In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train100k.csv')
test = pd.read_csv('test100k.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
np.mean(train['HasDetections'])

0.50156

In [4]:
np.mean(test['HasDetections'])

0.49981

In [5]:
#remove MachineIdentifier
train = train.drop(columns = ['MachineIdentifier'])
test = test.drop(columns = ['MachineIdentifier'])

In [6]:
## check for NA
## remove columns with NA greater than 50% of data (i.e. 50000 records)
drop_col_list = []
for i in train.columns:
    if train[i].isna().sum()>50000:
        drop_col_list.append(i)

In [7]:
train = train.drop(columns=drop_col_list)
test = test.drop(columns=drop_col_list)

In [8]:
cat_col = []
for i in train.columns:
    if np.issubdtype(train[i].dtype, np.number)==False:
        cat_col.append(i)

cat_col = cat_col+list(train.columns[train.columns.str.contains('Identifier')==True])

# drop possible duplicates
cat_col = list(set(cat_col))

### compute z-value for each category and create dummies accrdingly

In [9]:
temp = train[[*cat_col,'HasDetections']]
dummies_index = []
for col in cat_col:
    temp['HasDetections_total'] = temp['HasDetections']
    df = temp[[col,'HasDetections_total','HasDetections']].groupby(col).\
        agg({'HasDetections_total':'count','HasDetections':'mean'})
    df.columns = ['size','p^']
    df['SE'] = np.sqrt(df['p^']*(1-df['p^'])/df['size'])
    df['z-value'] = abs(df['p^']-0.5)/df['SE']
    dummies_index.append(df.index[(df['z-value'])>5&(df['size']>10)])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [10]:
dummies_names = []
for i in dummies_index:
    if len(i)>0:
        for ii in i:
            dummies_names.append(str(i.name)+'_'+str(ii))

In [11]:
len(dummies_names) 
# this did not consider if all categories one categorical variable are included
# however, algorithms will take care of perfect colinearity

45765

In [12]:
full_cat_data = train[[*cat_col]].append(test[[*cat_col]])

In [ ]:
%%time
dummies = pd.DataFrame(index=range(200000),columns=dummies_names)
for i in dummies_index:
    for ii in i:
        col_name = str(i.name)+'_'+str(ii)
        index_1 = full_cat_data[full_cat_data[i.name]==ii].index
        dummies.loc[index_1,col_name] = 1
dummies = dummies.fillna(0)

### gather train and test data

In [ ]:
train_x = dummies[:(100000)].reset_index(drop=True).\
    join(train.drop(columns = ['HasDetections',*cat_col]))
train_y = train['HasDetections']
test_x = dummies[100000:].reset_index(drop=True).\
    join(test.drop(columns = ['HasDetections',*cat_col]))
test_y = test['HasDetections']

In [ ]:
#fill na with median (only numerical field has na now)
train_x = train_x.fillna(value=train_x.median())
test_x = test_x.fillna(value=train_x.median())

In [ ]:
train_x.shape

### wrapper feature selection

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LogisticRegression

In [ ]:
%%time
lr = LogisticRegression(solver='lbfgs')
fs = SFS(lr, k_features=15, forward=True, floating=False, scoring='accuracy',cv=0)
fs = fs.fit(train_x, train_y)

In [ ]:
v_list = fs.k_feature_names_
v_list

In [ ]:
pd.Series(v_list).to_csv('v_list.csv',index=False)

## build models

In [ ]:
train_x_selected = train_x.loc[:,v_list]
test_x_selected = test_x.loc[:,v_list]

### logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lg = LogisticRegression(random_state=0).fit(train_x_selected, train_y)

In [ ]:
## training
lg.score(train_x_selected, train_y, sample_weight=None)

In [ ]:
## testing
lg.score(test_x_selected, test_y, sample_weight=None)

In [ ]:
lg_all = LogisticRegression(random_state=0).fit(train_x, train_y)

In [ ]:
## training
lg_all.score(train_x, train_y, sample_weight=None)

In [ ]:
## testing
lg_all.score(test_x, test_y, sample_weight=None)

### random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=500, max_depth = 15, random_state=0).fit(train_x_selected, train_y)

In [ ]:
## training
rfc.score(train_x_selected, train_y, sample_weight=None)

In [ ]:
## testing
rfc.score(test_x_selected, test_y, sample_weight=None)

### neural nets

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
nn = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(6), random_state=0).fit(train_x_selected, train_y)

In [ ]:
## training
nn.score(train_x_selected, train_y, sample_weight=None)

In [ ]:
## testing
nn.score(test_x_selected, test_y, sample_weight=None)

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3).fit(train_x_selected, train_y)

In [ ]:
## training
knn.score(train_x_selected, train_y, sample_weight=None)

In [ ]:
## testing
knn.score(test_x_selected, test_y, sample_weight=None)